In [ ]:
import sys 
sys.path.append(rf'C:\Users\simof\Documents\GitHub\thesis\data_processing')
from importlib import reload
import utils
reload(utils)
from utils import *

In [ ]:
os.listdir('data/popolazione')

In [ ]:
import pyodbc

mdb_file_path = "data/raw/population_data/Pop_comune_eta.mdb"
connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    f"DBQ={mdb_file_path};"
)

connection = pyodbc.connect(connection_string)
cursor = connection.cursor()

tables = cursor.tables(tableType='TABLE')
for table in tables:
    print(f"Table Name: {table.table_name}")


In [ ]:
# Load data into a DataFrame
query = "SELECT * FROM ISTAT_pop_comuni_fasce5" 
df = pd.read_sql(query, connection)

# Close connection
connection.close()

In [ ]:
df.info()

In [ ]:
df['Anno'].unique()

2002 - 2011

In [ ]:
df

In [ ]:
df2 = df.copy()
df2.loc[:,'Comune'] = ['A' + str(df2.loc[i, 'Comune']).zfill(6) for i in df2.index]

In [ ]:
comuni = pd.read_csv('data/clean_data/elenco_comuni_piemonte.csv')
comuni.head()

In [ ]:
pop_2002_2011 = df2[df2['Comune'].isin(comuni['Codice Comune'])]
pop_2002_2011.info()

In [ ]:
save_clean_data(pop_2002_2011, 'popolazione/population_2002_2011_per_age')

In [ ]:
pop_2002_2011.loc[:, 'Totale Eta'] = pop_2002_2011['Totale maschi'] + pop_2002_2011['Totale Femmine']

In [ ]:
for i, (id, g) in enumerate(pop_2002_2011.groupby(['Anno', 'Comune'])):
    if i > 0:
        break
    display(g)
    display(g.sum())

In [ ]:
for id, group in pop_2002_2011.groupby(['Anno', 'Comune']):
    pop_2002_2011.loc[group.index, 'Totale Comune Anno'] = group.sum()['Totale Eta']

In [ ]:
file = pop_2002_2011.drop(['Codice ISTAT Comune', 'Regione'], axis=1)
save_clean_data(file, 'popolazione/population_2002_2011_per_age')

---

2002-2019

In [ ]:
zip_path = 'data/raw/population_data'
extract_dir = 'data'

extract_zipfiles(zip_path, extract_dir)

In [ ]:
os.listdir('data/PopolazioneEta-SingolaArea-Comuni')[:4]

In [ ]:
pop_dict = {}
for code in comuni['Codice Comune'].str[1:]:
    file = [f for f in os.listdir('data/PopolazioneEta-SingolaArea-Comuni') if code in f]
    if len(file) > 0:
        pop_dict['A' + code] = pd.read_csv(f'data/PopolazioneEta-SingolaArea-Comuni/{file[0]}', skiprows=6, header=None, nrows=103, sep=';')

In [ ]:
comuni['Codice Comune'][~comuni['Codice Comune'].isin(list(pop_dict.keys()))]
# Moransengo-Tonengo missing
# È stato istituito il 1º gennaio 2023 dalla fusione di Moransengo e Tonengo.

In [ ]:
## RUN ONLY ONCE
for key, df in pop_dict.items():
    df.set_index(df.columns[0], drop=True, inplace=True)
    df.columns = df.iloc[-1,:]
    df.drop(df.index[-1], axis=0, inplace=True)
    df['Eta'] = df.index
    df['Comune'] = key
    

In [ ]:
pop_dict['A001001']

In [ ]:
result = pd.concat(pop_dict.values(), ignore_index=True)
result

In [ ]:
save_clean_data(result, 'popolazione/population_2002_2019_per_age')

---

1997-2001

In [ ]:
province = ['alessandria', 'asti', 'biella', 'cuneo', 'novara', 'torino', 'verbano', 'vercelli']
anni = ['97', '98', '99', '00', '01']
pop_dict = {}

for a in anni:
    pop_dict[a] = {}
    for p in province:
        pop_dict[a][p] = pd.read_csv(f'data/popolazione_1997_2001/popolazione_1997_2001/{p}_{a}.csv', header=0)

In [ ]:
## RUN ONLY ONCE
for a, pdf in pop_dict.items():
    for p, df in pdf.items():
        df.loc[:, 'Codice comune'] = ['A' + str(df.loc[i, 'Codice comune']).zfill(6) for i in df.index]
        df['Anno'] = f"19{a}" if a in ['97', '98', '99'] else f'20{a}'

In [ ]:
for a, pdf in pop_dict.items():
    pop_dict[a] = pd.concat(pdf.values(), ignore_index=True)

In [ ]:
result = pd.concat(pop_dict.values(), ignore_index=True)
result

In [ ]:
save_clean_data(result, 'popolazione/population_1997_2001_per_sex')

---

2019-2024

In [ ]:
anni = ['2019', '2020', '2021', '2022', '2023', '2024']
pop_dict = {}
for a in anni:
    pop_dict[a] = pd.read_csv(f'data/P2_{a}_it_Comuni/P2_{a}_it_Comuni.csv', header=1, sep=';')

In [ ]:
for a, df in pop_dict.items():
    df.loc[:, 'Codice comune'] = ['A' + str(df.loc[i, 'Codice comune']).zfill(6) for i in df.index]
    df.drop(df.columns[-1], inplace=True, axis=1)
    df['Anno'] = a

In [ ]:
result = pd.concat(pop_dict.values(), ignore_index=True)
result

In [ ]:
save_clean_data(result.drop(result.index[-1], axis=0), 'popolazione/population_2019_2024_per_sex')

---

Select only cities of Piemonte

In [ ]:
elenco = pd.read_csv('data/clean_data/elenco_comuni_piemonte.csv')
elenco.head()

In [ ]:
os.listdir('data/clean_data/popolazione')

In [ ]:
df1 = pd.read_csv('data/clean_data/popolazione/population_1997_2001_per_sex.csv')
df2 = pd.read_csv('data/clean_data/popolazione/population_2002_2011_per_age.csv')
df3 = pd.read_csv('data/clean_data/popolazione/population_2002_2019_per_age.csv')
df4 = pd.read_csv('data/clean_data/popolazione/population_2019_2024_per_sex.csv')

In [ ]:
pop1 = df1[df1['Codice comune'].isin(elenco['Codice Comune'])]
pop2 = df2[df2['Comune'].isin(elenco['Codice Comune'])]
pop3 = df3[df3['Comune'].isin(elenco['Codice Comune'])]
pop4 = df4[df4['Codice comune'].isin(elenco['Codice Comune'])]

In [ ]:
save_clean_data(pop2, 'popolazione_piemonte/population_2002_2011_per_age.csv')
save_clean_data(pop3, 'popolazione_piemonte/population_2002_2019_per_age.csv')
save_clean_data(pop4, 'popolazione_piemonte/population_2019_2024_per_sex.csv')

In [ ]:
pop1[pop1['Comune'].isna()]

In [ ]:
save_clean_data(pop1, 'popolazione_piemonte/population_1997_2001_per_sex.csv')

---